In [2]:
import importlib
import xarray as xr
import numpy as np
import pandas as pd
from CASutils import filter_utils as filt
from CASutils import calendar_utils as cal
import sys

importlib.reload(filt)
importlib.reload(cal)

<module 'CASutils.calendar_utils' from '/home/islas/python/CASanalysis/CASutils/calendar_utils.py'>

In [3]:
data = xr.open_dataset("/project/cas/islas/savs/python/ISD/T2M_ISD_1979_2014.nc")
t2m = data['t2m']
lon = data['lon']
lat = data['lat']

In [6]:
# convert to celsius !!! I should change this to do it at the read stage
t2m = (t2m - 32)/1.8

In [9]:
# only use the stations that have data on more than 80% of the days
numnans = [np.count_nonzero(np.isnan(t2m[i,:])) for i in range(0,t2m['station'].size,1)]
mask = (np.array(numnans) < (0.8*t2m.time.size))
t2m = t2m[mask,:] ; lon = lon[mask] ; lat = lat[mask]

In [10]:
# calculate the seasonal cycle for each station and subtract to produce anomalies.  
t2mseason = t2m.groupby('time.dayofyear').mean('time', skipna=True)
t2m4harm = filt.calc_season_nharm(t2mseason, 4, dimtime=1)
anoms = t2m.groupby('time.dayofyear') - t2m4harm

you got nans


In [11]:
# group by djfs and subtract the DJF mean
djfanoms = cal.group_season_daily(anoms, 'DJF')
djfmean = djfanoms.mean('day', skipna=True)
djfanoms = djfanoms - djfmean

moving time axis to the start


In [12]:
isdvar = np.var(djfanoms, axis=(0,1))
isdvar = isdvar.rename("isdvar")

/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xarray/core/nputils.py:227: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


In [14]:
isdvar.to_netcdf("/project/cas/islas/savs/python/snowpaper/DATA_SORT/deseasonalized_tvar/TVAR_ISD.nc")
lon.to_netcdf("/project/cas/islas/savs/python/snowpaper/DATA_SORT/deseasonalized_tvar/TVAR_ISD.nc", mode="a")
lat.to_netcdf("/project/cas/islas/savs/python/snowpaper/DATA_SORT/deseasonalized_tvar/TVAR_ISD.nc", mode="a")